In [1]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
class Net(nn.Module):
    def __init__(self):
        #nn.Mudule子类的函数必须在构造函数中执行父类的构造函数
        #下式等价于nn.Module.__init__(self)
        super(Net,self).__init__()
        #1为输入通道数，6为输出通道数，5为卷积核大小
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        #全连接层
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        #reshape 
        x = x.view(x.size()[0],-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
# 查看参数
params = list(net.parameters())
print(len(params))


10


In [5]:
for name,parameters in net.named_parameters():
    print(name,':',parameters.size())

conv1.weight : torch.Size([6, 1, 5, 5])
conv1.bias : torch.Size([6])
conv2.weight : torch.Size([16, 6, 5, 5])
conv2.bias : torch.Size([16])
fc1.weight : torch.Size([120, 400])
fc1.bias : torch.Size([120])
fc2.weight : torch.Size([84, 120])
fc2.bias : torch.Size([84])
fc3.weight : torch.Size([10, 84])
fc3.bias : torch.Size([10])


In [6]:
# 只有Variable 才有自动求导功能
input = Variable(t.rand(1,1,32,32))
out = net(input)
print(out.size())

torch.Size([1, 10])


In [7]:
#所有参数的梯度清零
net.zero_grad() # 所有参数的梯度清零
out.backward(t.ones(1,10)) # 反向传播

#### 损失函数

In [8]:
output = net(input)
target = t.arange(0.,10.).view(1,10) 
criterion = nn.MSELoss()
loss = criterion(output, target)
loss # loss是个scalar

tensor(28.6031, grad_fn=<MseLossBackward>)

In [9]:
loss.grad_fn

#### 梯度变化

In [10]:
net.zero_grad()
print('反向传播前conv1.bias梯度')
print(net.conv1.bias.grad)
loss.backward()
print('反向传播后的梯度')
print(net.conv1.bias.grad)

反向传播前conv1.bias梯度
tensor([0., 0., 0., 0., 0., 0.])
反向传播后的梯度
tensor([ 0.0012, -0.0028,  0.1404, -0.0688,  0.0024, -0.0217])


#### 优化器

In [11]:
import torch.optim as optim
optimizer  = optim.SGD(net.parameters(),lr = 0.01)

#在训练过程中 先八梯度清零 和net.zero_grad()效果一样
optimizer.zero_grad()

#计算损失
output = net(input)
loss = criterion(output,target)

#反向传播
loss.backward()
#更新参数
optimizer.step()